In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

- calculate and updata theta by gradien descent method
$$J(\theta) = \frac{1}{m} \overset{m}{\underset{i=1}{\sum}}[y_i log(h_{\theta}(x_i)) + (1-y_i)log(1-h_{\theta}(x_i)]$$
$$\theta_j := \theta_j -  \frac{\alpha}{m}\overset{m}{\underset{i=1}{\sum}}(h_{\theta}(x_i) - y_i)x_{ij}$$

In [6]:
def gradient_descent_logit():
    # input values
    data_name=input("Enter the name of data file [(ex) sample1.csv] : ") # data name
    coding_fm=int(input("Select the data Seperator(1 = ' ' or 2 = ','): ")) # data separator of training data
    separator_fm={coding_fm ==1 : " "}.get(True, ",")
    header=input("Does the data have column header? (y/n) : ")
    if(header=="y") :
        trdata=pd.read_csv(data_name, sep=separator_fm) # loading data with header
        res_pos = input(f'Enter the column name of response variable among {list(trdata.columns)} : ')
        y = trdata[res_pos]
        X = trdata.drop(res_pos, axis = 1)

    else : 
        trdata=pd.read_csv(data_name, sep=separator_fm, header=None) # loading data without header
        res_pos = int(input(f'Enter the column position of the response variable : \n [from 1 to {trdata.shape[1]}] : '))
        X_index = []
        for i in range(len(trdata.columns)): 
            if i == res_pos-1 : continue
            X_index.append(i)

        # define response variable and predictor
        y = trdata.iloc[:,res_pos-1]
        X = trdata.iloc[:,X_index]
    
    theta = [float(x) for x in input(f'Enter {X.shape[1] + 1} number of starting point of gradient descent : '
                                    ).split(',')]
    learning_rate = float(input('Enter the learning rate of gradient descent between 0 and 1 : '))
    max_iter = int(input('Enter the max_iteration limit : '))
    threshold = float(input('Enter the threshold of gradient descent : '))
    out_name=input("Enter the output file name to export [(ex) result.txt] : ")
    
    # add constant in X variable
    X.insert(loc = 0 , column= 'constant', value = np.ones(trdata.shape[0]))
    
    
    theta_history = pd.DataFrame(theta)
    # gradient descent
    for iter in range(1,max_iter):
        ypred = 1/(1+np.exp(-X@theta))
        update = learning_rate * (1/len(X)) * ((ypred - y).transpose() @ X)
        theta = np.array(theta - update)
        theta_history.insert(loc = iter, column = f'{iter}', value = theta)
        if np.all(np.abs(update) <= threshold):
            break
            
    # sm logistic regression
    mod = sm.Logit(y,X)
    res = mod.fit()
    
    # write file
    f = open(out_name,'w+')
    f.write('Coefficients by Gradient Descent Method\n')
    f.write('-------------\n')
    f.write(f'Constant: {round(theta[0],3)}\n')
    for i in range(1,len(theta)):
        f.write(f'Beta {i} : {round(theta[i],3)}\n')

    f.write('Coefficients by STATMODELS \n')
    f.write('-------------\n')
    f.write(f'Constant: {round(list(res.params)[0],3)}\n')
    for i in range(1,len(res.params)):
        f.write(f'Beta {i} : {round(list(res.params)[i],3)}\n')
    f.close()

    # print results
    print('------------------------')
    print('file has been saved')
    print('result is\n\n')
    
    r = open(out_name,'r')
    for line in r.readlines():
        print(line)
    r.close()
    return {'gradient_decent_beta' : theta,'history' : theta_history, 'OLS_beta' : res.params}

In [7]:
result = gradient_descent_logit()

Enter the name of data file [(ex) sample1.csv] :  sample1.csv
Select the data Seperator(1 = ' ' or 2 = ','):  2
Does the data have column header? (y/n) :  y
Enter the column name of response variable among ['x1', 'x2', 'y'] :  y
Enter 3 number of starting point of gradient descent :  0,0,0
Enter the learning rate of gradient descent between 0 and 1 :  0.3
Enter the max_iteration limit :  10000
Enter the threshold of gradient descent :  0.0001
Enter the output file name to export [(ex) result.txt] :  result.txt


Optimization terminated successfully.
         Current function value: 0.283800
         Iterations 8
------------------------
file has been saved
result is


Coefficients by Gradient Descent Method

-------------

Constant: 0.166

Beta 1 : 2.8

Beta 2 : 2.796

Coefficients by STATMODELS 

-------------

Constant: 0.168

Beta 1 : 2.822

Beta 2 : 2.819

